# Putting it all together
- https://huggingface.co/learn/nlp-course/chapter2/6?fw=pt

</br>

지난 section들에서, pipeline이 어떻게 동작하는지, 대부분의 것들을 직접 해보았다.  
- tokenize가 어떻게 진행되고, tokenization결과가 무엇인지 보았다.
- token을 input ID로 바꾸어보았다.
- padding을 해보았다.
- truncation을 보았다.
- attention mask를 보았다.

하지만, section 2에서 본 것처럼, Transformer API는 이 모든 것들을 높은 추상화 단계의 function으로 다룰 수 있다. `tokenizer`를 seuqence에 직접적으로 호출하면 model에 사용 가능한 input을 반환받는다.



In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

- distilbert-base-uncased-finetuned-sst-2-english -> 검색해보니까 github같이 model 자체의 이름이 이거였다. (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
- hugging face에 배포된 model 이름을 그대로 붙여넣으면 사용이 가능한 것 같다.

여기서, model_inputs 변수는 모델이 작동하기 위해 필요한 모든 것을 가지고있다.  
DistilBERT에 대해서, 이는 input ID와 attention mask를 가지고있다.  
추가적인 input을 허용하는 다른 모델들 역시 tokenizer object에 의한 output을 가지고있다.

이 method는 powerful하다. 먼저, 이는 single sequence를 tokenize할 수 있다. 아래가 해당 예시이다.

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs)

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


이번엔 multiple sequence를 변환해보자.

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)
model_inputs

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

- 각 문장 그대로 tokenization이 되었다.

위의 multiple sequence예시에 padding을 추가해보자.

In [ ]:
# 제일 길이가 긴 sequence에 맞춰서 padding
model_inputs = tokenizer(sequences, padding="longest")
print(model_inputs)

# 모델이 다룰 수 있는 최대 길이까지 padding
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")
print(model_inputs)

# 특정 길이의 sequnece가 될때까지 padding
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)
print(model_inputs)


{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

이번엔 truncate를 해보자.

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# 모델이 다룰 수 있는 최대 길이를 넘어가면 trucate
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)
print(model_inputs)

# 특정 길이까지 truncate
model_inputs = tokenizer(sequences, max_length=8, truncation=True)
print(model_inputs)

{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}
{'input_ids': [[101, 1045, 1005, 2310, 2042, 3403, 2005, 102], [101, 2061, 2031, 1045, 999, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


`tokenizer` object는 특정 framework의 tensor로 변환시킬 수도 있다. 즉, 바로 모델의 input으로 사용할 수 있게 해준다는 의미이다.  
예를 들어, 아래의 간단한 코드에서, 우리는 toeknizer가 다른 framework의 tensor를 반환하도록 할 수 있다. 만약 pytorch framework라면 `pt`를 통해서 pytorch tensor를 반환할 수 있다. 그 외에 tensorflow나 numpy array의 tensor도 반환이 가능하다.

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")


## Special tokens

우리가 tokenizer로 input ID를 반환받으면 우리가 이전에 봤던 것과 결과가 조금 다름을 알 수 있다.

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)


[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


보면 시작과 끝에 token id가 한 개씩 추가되었음을 알 수 있다. 한 번, 이 둘을 decode 해보자.

In [ ]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))


[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


tokenizer는 문장 처음에 [CLS]와 문장 끝에 [SEP]를 추가하였다. 이는 모델이 pre training 될 때 이들과 학습되었기 때문이다. 따라서 inference에서 동일한 결과를 얻기 위해 이들을 추가할 필요가 있다. 하지만 이런 special token들을 추가하지 않거나 끝에 하나만 추가하는 모델들도 몇몇 있음을 알아두자.


## 정리: Tokenizer부터 Model까지

우리는 tokenizer부터 각각의 step들을 보았다. 이제 multiple seuqence와 매우 긴 sequence를 API를 통해 어떻게 다루는지 보자.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)